In [1]:
import argparse
import sys
import torch
import numpy as np
import os

In [2]:
sys.path.append('./base_model')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from base_model.dataset import wrap_dataset
from base_model.model import DisentangleVAE
from interface import PolyDisVAE
from amc_dl.torch_plus.train_utils import get_zs_from_dists

In [4]:
def load_model(device=device):
    interface = PolyDisVAE.init_model(device=device)
    interface.load_model('./model_param/polydis-v1.pt')
    model = DisentangleVAE('disvae', interface.device,
                           interface.chd_encoder,
                           interface.txt_encoder,
                           interface.pnotree_decoder,
                           interface.chd_decoder)
    model.eval()
    return model

In [5]:
load_model()

/workspace/vae-tuning-texutres/interface.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dic = torch.load(model_path, map_location=self.device)


DisentangleVAE(
  (chd_encoder): ChordEncoder(
    (gru): GRU(36, 1024, batch_first=True, bidirectional=True)
    (linear_mu): Linear(in_features=2048, out_features=256, bias=True)
    (linear_var): Linear(in_features=2048, out_features=256, bias=True)
  )
  (rhy_encoder): TextureEncoder(
    (cnn): Sequential(
      (0): Conv2d(1, 10, kernel_size=(4, 12), stride=(4, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    )
    (fc1): Linear(in_features=290, out_features=1000, bias=True)
    (fc2): Linear(in_features=1000, out_features=256, bias=True)
    (gru): GRU(256, 1024, batch_first=True, bidirectional=True)
    (linear_mu): Linear(in_features=2048, out_features=256, bias=True)
    (linear_var): Linear(in_features=2048, out_features=256, bias=True)
  )
  (decoder): PianoTreeDecoder(
    (note_embedding): Linear(in_features=135, out_features=128, bias=True)
    (z2dec_hid_linear): Linear(in_features=512, out_feature

In [6]:
def prepare_tensors(x, c, pr_mat, device):
    x = torch.tensor(x).long().unsqueeze(0).to(device)
    c = torch.tensor(c).float().unsqueeze(0).to(device)
    pr_mat = torch.tensor(pr_mat).float().unsqueeze(0).to(device)
    return x, c, pr_mat

In [7]:
def run_with_latents(model, x, c, pr_mat):
    embedded_x, lengths = model.decoder.emb_x(x)
    dist_chd = model.chd_encoder(c)
    dist_rhy = model.rhy_encoder(pr_mat)
    z_chd, z_txt = get_zs_from_dists([dist_chd, dist_rhy], True)
    dec_z = torch.cat([z_chd, z_txt], dim=-1)
    recon_pitch, recon_dur = model.decoder(dec_z, False, embedded_x, lengths, 0., 0.)
    recon_root, recon_chroma, recon_bass = model.chd_decoder(z_chd, False, 0., c)
    return recon_pitch, recon_dur, dist_chd, dist_rhy, recon_root, recon_chroma, recon_bass, z_chd, z_txt

In [8]:
def compute_segment(model, x, c, pr_mat):
    recon_pitch, recon_dur, dist_chd, dist_rhy, recon_root, recon_chroma, recon_bass, z_chd, z_txt = run_with_latents(model, x, c, pr_mat)
    loss_values = model.loss_function(x, c, recon_pitch, recon_dur,
                                      dist_chd, dist_rhy,
                                      recon_root, recon_chroma, recon_bass,
                                      beta=0.1, weights=(1, 0.5))
    total_loss, _, _, _, kl_loss, kl_chd, kl_rhy, _, _, _, _ = loss_values
    return z_chd.squeeze(0), z_txt.squeeze(0), kl_loss, kl_chd, kl_rhy, total_loss

In [12]:
def compute_losses(npz_path, output_dir="./losses"):

    # Garante que o diretório de saída existe
    os.makedirs(output_dir, exist_ok=True)
    
    # Extrai nome base do arquivo (sem caminho, sem extensão)
    base_name = os.path.splitext(os.path.basename(npz_path))[0]
    
    model = load_model(device=device)
    dataset = wrap_dataset([npz_path], [0], shift_low=0, shift_high=0,
                           num_bar=2, contain_chord=True)

    results = []
    
    for idx in range(len(dataset)):
    
        
        _, _, pr_mat, x, c, _ = dataset[idx]
        x_t, c_t, pr_mat_t = prepare_tensors(x, c, pr_mat, model.device)
        z_chd, z_txt, kl_loss, kl_chd, kl_rhy, total_loss = compute_segment(
            model, x_t, c_t, pr_mat_t)



        # Monta o nome do arquivo de saída
        segment_name = f"{base_name}-{idx+1:03d}"  # Ex: 001-001
        save_path = os.path.join(output_dir, f"{segment_name}.npz")

        #Salva os dados em arquivo .npz
        np.savez_compressed(save_path,
                            z_chd=z_chd.detach().cpu().numpy(),
                            z_txt=z_txt.detach().cpu().numpy(),
                            kl_loss=kl_loss.item(),
                            kl_chd=kl_chd.item(),
                            kl_rhy=kl_rhy.item(),
                            final_loss=total_loss.item())
        
        #pdb.set_trace()
        #results.append({
            #'segment': idx,
         #   'name': segment_name,
          #  'z_chd': z_chd.detach().cpu().numpy(),
           # 'z_txt': z_txt.detach().cpu().numpy(),
            #'kl_loss': kl_loss.item(),
           # 'kl_chd': kl_chd.item(),
            #'kl_rhy': kl_rhy.item(),
            #'final_loss': total_loss.item(),
       # })
    #return results

In [13]:
def main(path):
    """Compute per-segment losses for the given NPZ file.

    Parameters
    ----------
    path : str
        Path to the ``.npz`` file.

    Returns
    -------
    list[dict]
        A list with one entry per segment containing ``z_chd``, ``z_txt`` and
        all loss values.
    """
    results = compute_losses(path)
    return results

In [14]:
pwd

'/workspace/vae-tuning-texutres'

In [15]:
main( "./dataSet/POP09-PIANOROLL-4-bin-quantization/001.npz")

72


```
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Compute loss for one NPZ file')
    parser.add_argument('npz_path', help='Path to NPZ file')
    args = parser.parse_args()
    main(args.npz_path)
```